# Random Forest

## Inizializzazione variabili

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
import matplotlib.pyplot as plt

In [79]:
# Percorso dove salvare l'excel
excel_path = "Excel/RandomForest_Results.xlsx"

# Genera il dataset
parte1 = pd.read_csv("Excel/Parte1.csv")
parte2 = pd.read_csv("Excel/parte2_v4.csv")

# Viene rinominata la feature ID in Patiene_ID per perettere il merge
parte2 = parte2.rename(columns={"ID": "Patient_ID"})

# Viene fatto un inner‐merge su Patient_ID:
df = parte1.merge(parte2, on="Patient_ID", how="inner")
df = df.drop(columns=["Patient_ID"])

# Dividi X e Y
Y = df['Has_Diagnostics']
X = df.drop(columns=['Has_Diagnostics'])

# Funzione per metriche
def get_metrics(y_true, y_predicted):
    matrix = confusion_matrix(y_true, y_predicted)
    accuracy = accuracy_score(y_true, y_predicted)
    precision = precision_score(y_true, y_predicted, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_predicted, average='weighted', zero_division=0)
    f1score = f1_score(y_true, y_predicted, average='weighted', zero_division=0)
    return matrix, accuracy, precision, recall, f1score

### Riempimento valori null con la media

In [80]:
df_media = df.fillna(df.mean(numeric_only=True))

# Dividi X e Y
del X, Y
Y = df_media['Has_Diagnostics']
X = df_media.drop(columns=['Has_Diagnostics'])

### Riempimento valori null con la mediana

In [193]:
df_mediana = df.fillna(df.median(numeric_only=True))

# Dividi X e Y
del X, Y
Y = df_mediana['Has_Diagnostics']
X = df_mediana.drop(columns=['Has_Diagnostics'])

### Riempimento valori null con KNNImputer

In [98]:
# Rimuovi momentaneamente X e Y
df_temp = df.copy()

# Crea un imputatore KNN
imputer = KNNImputer(n_neighbors=5)

# Applica l'imputazione solo sulle colonne numeriche
df_imputed_array = imputer.fit_transform(df_temp.select_dtypes(include='number'))

# Ricrea il DataFrame con le stesse colonne
df_imputed = pd.DataFrame(df_imputed_array, columns=df_temp.select_dtypes(include='number').columns)

# Se ci sono colonne non numeriche, le aggiungiamo di nuovo (senza modificarle)
for col in df_temp.columns:
    if col not in df_imputed.columns:
        df_imputed[col] = df_temp[col]

# Dividi X e Y
del X, Y
Y = df_imputed['Has_Diagnostics']
X = df_imputed.drop(columns=['Has_Diagnostics'])

### Riempimento valori null con IterativeImputer

In [220]:
# Rimuovi momentaneamente X e Y
df_temp = df.copy()

# Crea un imputatore iterativo
imputer = IterativeImputer(max_iter=10, random_state=42)

# Applica l'imputazione solo sulle colonne numeriche
df_imputed_array = imputer.fit_transform(df_temp.select_dtypes(include='number'))

# Ricrea il DataFrame con le stesse colonne
df_imputed = pd.DataFrame(df_imputed_array, columns=df_temp.select_dtypes(include='number').columns)

# Se ci sono colonne non numeriche, le aggiungiamo di nuovo (senza modificarle)
for col in df_temp.columns:
    if col not in df_imputed.columns:
        df_imputed[col] = df_temp[col]

# Dividi X e Y
del X, Y
Y = df_imputed['Has_Diagnostics']
X = df_imputed.drop(columns=['Has_Diagnostics'])

In [111]:
Prova = 24
n_estimators = 300
max_depth = 5

## Metodo dell 80/20

In [112]:
print("=== VALIDAZIONE 80/20 ===")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

cm, acc, prec, rec, f1 = get_metrics(y_test, y_pred)

#print(f"Accuracy: {acc:.4f}")
#print(f"Precision: {prec:.4f}")
#print(f"Recall: {rec:.4f}")
#print(f"F1-Score: {f1:.4f}")

#disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
#disp.plot(cmap=plt.cm.Blues)
#plt.title("Confusion Matrix 80/20")
#plt.show()

# Calcola importanza delle feature
importances = clf.feature_importances_
features = X.columns
importances_dict = {f'Imp_{feat}': [imp] for feat, imp in zip(features, importances)}

# Prepara il DataFrame da salvare
df_split = pd.DataFrame({
    'Prova': [Prova],
    'n_estimators': [n_estimators],
    'max_depth': [max_depth],
    'Accuracy': [acc],
    'Precision': [prec],
    'Recall': [rec],
    'F1-Score': [f1],
    'Confusion_Matrix': [cm.tolist()],
    ' ': [None]
} | importances_dict)  # Unione dizionari

# Aggiungi a eventuali dati già presenti
if os.path.exists(excel_path):
    with pd.ExcelFile(excel_path) as reader:
        if 'Split_80_20_Test' in reader.sheet_names:
            prev_data = pd.read_excel(reader, sheet_name='Split_80_20_Test')
            df_split = pd.concat([prev_data, df_split], ignore_index=True)

# Scrivi sul file
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_split.to_excel(writer, sheet_name='Split_80_20_Test', index=False)

print("✅ Risultati 80/20 salvati su Excel.")

=== VALIDAZIONE 80/20 ===
✅ Risultati 80/20 salvati su Excel.


## K-Fold cross recognition

In [113]:
print("=== VALIDAZIONE K-FOLD ===")

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for i, (train_index, test_index) in enumerate(kf.split(X, Y), start=1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    cm, acc, prec, rec, f1 = get_metrics(y_test, y_pred)

#    print(f"\n--- Fold {i} ---")
#    print(f"Accuracy: {acc:.4f}")
#    print(f"Precision: {prec:.4f}")
#    print(f"Recall: {rec:.4f}")
#    print(f"F1-Score: {f1:.4f}")

#    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
#    disp.plot(cmap=plt.cm.Blues)
#    plt.title(f"Confusion Matrix Fold {i}")
#    plt.show()

    # Calcola importanza delle feature
    importances = clf.feature_importances_
    features = X.columns
    importances_dict = {f'Imp_{feat}': imp for feat, imp in zip(features, importances)}

    # Salva tutti i dati in un'unica riga
    fold_results.append({
        'Prova': Prova,
        'KFold' : i,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Confusion_Matrix': cm.tolist(),
        ' ': None,  # cella vuota per separare le metriche dalle importances
        **importances_dict
    })

df_kfold = pd.DataFrame(fold_results)

# Aggiungi a eventuali dati già presenti
if os.path.exists(excel_path):
    with pd.ExcelFile(excel_path) as reader:
        if 'KFold_CV_Test' in reader.sheet_names:
            prev_kfold = pd.read_excel(reader, sheet_name='KFold_CV_Test')
            df_kfold = pd.concat([prev_kfold, df_kfold], ignore_index=True)

# Scrivi sul file
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_kfold.to_excel(writer, sheet_name='KFold_CV_Test', index=False)

print("✅ Risultati K-Fold salvati su Excel.")

=== VALIDAZIONE K-FOLD ===
✅ Risultati K-Fold salvati su Excel.


## TRAIN-FOLD / TEST-HOLDOUT validation

In [114]:
print("=== VALIDAZIONE TRAIN-FOLD / TEST-HOLDOUT ===")
val_results = []
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

for i, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
    X_fold_train = X_train.iloc[train_idx]
    y_fold_train = y_train.iloc[train_idx]

    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    clf.fit(X_fold_train, y_fold_train)

    # Si testa sempre sullo stesso 20%
    y_pred = clf.predict(X_test)

    cm, acc, prec, rec, f1 = get_metrics(y_test, y_pred)

#    print(f"\n--- Fold {i} (train su fold, test fisso) ---")
#    print(f"Accuracy: {acc:.4f}")
#    print(f"Precision: {prec:.4f}")
#    print(f"Recall: {rec:.4f}")
#    print(f"F1-Score: {f1:.4f}")

#    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
#    disp.plot(cmap=plt.cm.Blues)
#    plt.title(f"Confusion Matrix Fold {i}")
#    plt.show()

    # Feature importances
    importances = clf.feature_importances_
    importances_dict = {f'Imp_{feat}': imp for feat, imp in zip(X.columns, importances)}

    # Salva i risultati del fold
    val_results.append({
        'Prova': Prova,
        'KFold' : i,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'Confusion_Matrix': cm.tolist(),
        ' ': None,  # cella vuota per separare le metriche dalle importances
        **importances_dict
    })

# Crea il DataFrame
df_nested = pd.DataFrame(val_results)

# Se esiste già, aggiungi i dati al foglio
if os.path.exists(excel_path):
    with pd.ExcelFile(excel_path) as reader:
        if 'TrainFold_TestFixed_Test' in reader.sheet_names:
            prev_nested = pd.read_excel(reader, sheet_name='TrainFold_TestFixed_Test')
            df_nested = pd.concat([prev_nested, df_nested], ignore_index=True)
            
# Salva su Excel
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_nested.to_excel(writer, sheet_name='TrainFold_TestFixed_Test', index=False)

print("✅ Risultati nested fold salvati su Excel.")

=== VALIDAZIONE TRAIN-FOLD / TEST-HOLDOUT ===
✅ Risultati nested fold salvati su Excel.


# Creazione dataset test per complicanze

In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1) Vengono letti i due file
parte1 = pd.read_csv("Excel/Parte1.csv")
parte2 = pd.read_csv("Excel/parte2_v1.csv")

# 1.1) Viene rinominata la feature ID in Patiene_ID per perettere il merge
parte2 = parte2.rename(columns={"ID": "Patient_ID"})

# 1.2) Viene fatto un inner‐merge su Patient_ID:
clust = parte1.merge(parte2, on="Patient_ID", how="inner")

# 2) Viene letto Diagnostics.csv per ricavare i codici associati a ciascun paziente
diag = pd.read_csv("Excel/Diagnostics.csv")

# 2.1) Per ogni paziente, viene prenso il Code con il maggior numero di occorrenze
diag_unique = (
    diag
    .groupby("Patient_ID")["Code"]
    .agg(lambda codes: codes.value_counts().idxmax())
    .reset_index()
    .rename(columns={"Code": "CodeStrat"})
)

# 2.2) Se hai codici troppo rari vengono raggruppati sotto "RARE" per evitare errori di stratify
code_counts = diag_unique["CodeStrat"].value_counts()
rare_codes = set(code_counts[code_counts < 2].index)
diag_unique["CodeStrat"] = diag_unique["CodeStrat"].apply(
    lambda c: "RARE" if c in rare_codes else c
)

# 3) Suddividi i soli pazienti “diagnosticati” 80/20, stratificando su CodeStrat
diag_train_patients, diag_test_patients = train_test_split(
    diag_unique["Patient_ID"],
    test_size=0.20,
    random_state=42,
    stratify=diag_unique["CodeStrat"]
)

# 4) Trova i pazienti in clust che NON compaiono in diag_unique (ossia “senza diagnosi”)
all_clust_patients = set(clust["Patient_ID"].unique())
diag_patients = set(diag_unique["Patient_ID"].unique())
no_diag_patients = list(all_clust_patients - diag_patients)

# 5) Dividi i pazienti “senza diagnosi” (%) 80/20 in modo casuale
no_diag_train, no_diag_test = train_test_split(
    no_diag_patients,
    test_size=0.20,
    random_state=42
)

# 6) Costruisci la lista finale di test (diagnosticati nel 20% + senza diagnosi nel 20%)
test_patients  = set(diag_test_patients)  | set(no_diag_test)
train_patients = set(diag_train_patients) | set(no_diag_train)

# Verifica che non ci siano sovrapposizioni
assert train_patients.isdisjoint(test_patients), "Errore: un paziente è in entrambi i set!"

# 7) Filtra clust in base a Patient_ID: 
#    - se Patient_ID ∈ test_patients → va in df_test
#    - altrimenti va in df_train
df_train = clust[~clust["Patient_ID"].isin(test_patients)].copy()
df_test  = clust[ clust["Patient_ID"].isin(test_patients)].copy()

# 8) Rimuovi di nuovo la colonna Patient_ID, come richiesto, prima di salvare
df_train_final = df_train.drop(columns=["Patient_ID"])
df_test_final  = df_test.drop(columns=["Patient_ID"])

# 9) Salva i due file risultanti
df_train_final.to_csv("Excel/Classification_train.csv", index=False)
df_test_final.to_csv("Excel/Classification_test.csv",  index=False)

print("Suddivisione completata:")
print(f"  - Train (Classification_train.csv): {len(df_train_final)} righe")
print(f"  - Test  (Classification_test.csv):  {len(df_test_final)}  righe")

Suddivisione completata:
  - Train (Classification_train.csv): 581 righe
  - Test  (Classification_test.csv):  142  righe
